In [1]:
# Import Statements
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
from std_msgs.msg import Int32MultiArray as Array
from std_msgs.msg import MultiArrayDimension as Dimension
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String
import EnvMap
import Buoy_List
from numba import jit

In [2]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub_array = rospy.Publisher('updated_array', Array, queue_size=10)
pub_map = rospy.Publisher('updated_map', Map, queue_size=10)

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0
b_list = []
tri_width = 5
dist_error = 0.15
map_height = 0
camera_range = 85/map_resolution

overlay = True
display_robot_yaw = False
display_rect = True
display_map = True
display_info = False
visualization = False
publish_map_or_nah = True

np.set_printoptions(threshold=sys.maxsize)

In [3]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [4]:
def disp_map(map_img):
    map_img = np.swapaxes(map_img, 0, 1)
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [5]:
def transform(img):

    global bearings
    global areas
    global tags    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    image = pp(img,width,height)
    img2= dbscan(image, img)

    return img2

def pp(image, width, height):

    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,
                                        np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),
                                        np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] \
                                        = [0,0,0]
    return image

def dbscan(image, original):

    img = image[:,:,2]
    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)
    nonzero = np.nonzero(thresh1)
    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])
    X=np.column_stack((xp,yp))
    if len(X) > 0:
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_
        array = np.column_stack((X,labels))
        sort = array[np.argsort(array[:, 2])]
        sort = sort[np.logical_not(sort[:,2] == -1)]      
        if len(sort) > 0:
            points = sort[:,[0,1]]
            tb = None
            first_locations = np.delete(np.nonzero(np.r_[1,np.diff(sort[:,2])[:-1]]),0)
            points_split = np.split(points,np.reshape(first_locations,-1))
            for clusters in np.arange(len(points_split)).tolist():
                tb = rectangle(points_split[clusters],image,original)
            original = tb
    return original


In [6]:
def rectangle(mat,image,original):
    
    global overlay
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    
    tag = 0
    if overlay:
        if width < height < 4*width and width*height > 45:
            tag, original = identify_buoy_overlay(mat,image,original,x,y,width,height,min_rect,tag)
        if tag == 0:
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if width < height < 4*width and width*height > 45:
            image = identify_buoy_hsv(mat,image,x,y, width,height,min_rect,tag)
        return image

def identify_buoy_overlay(mat,image,original,x,y,width,height,min_rect,tag):
    
    global bearings
    global areas
    global tags
    
    font = cv.FONT_HERSHEY_SIMPLEX
    color = color_hsv(mat,image)
    if color =='green':
        tag = 2
        cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
        tags.append(tag)
    elif color =='red':
        tag = 3
        cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
        tags.append(tag)
    else:
        tag = 1
        cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
    bearing = (min_rect[0][0]-640.0)/1280.0*80.0
    cv.putText(original,'bearing: '+ str(round(bearing,2)),(np.int0(x+width/2),np.int0(y+height/2.5)), font, 0.5,(0,0,0),2,cv.LINE_AA)
    bearings.append(bearing)
    area = min_rect[1][0]*min_rect[1][1]
    areas.append(area)
    
    return tag, original

def identify_buoy_hsv(mat,image,x,y,width,height,min_rect,tag):
    
    global bearings
    global areas
    global tags
    
    font = cv.FONT_HERSHEY_SIMPLEX
    color = color_hsv(mat,image)
    
    if color =='green':
        tag = 2
        cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    elif color =='red':
        tag = 3
        cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    else:
        tag = 1
        cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    
    cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
    bearing = (min_rect[0][0]-640.0)/1280.0*80.0
    bearings.append(bearing)
    area = min_rect[1][0]*min_rect[1][1]
    areas.append(area)
    
    return image
    
def color_hsv(mat, image):

    sum_hue = 0
    sum_sat = 0
    sum_val = 0
    mat_add = 0
#     print len(mat)
    for x in range(len(mat)):
        if image [mat[x][1]] [mat[x][0]] [2] > 5:
            mat_add = mat_add+1
            sum_val += image [mat[x][1]] [mat[x][0]] [2]
            sum_sat += image [mat[x][1]] [mat[x][0]] [1]
            sum_hue += image [mat[x][1]] [mat[x][0]] [0]
            
    if mat_add > 0:
        sat = sum_sat/mat_add
        val = sum_val/mat_add
        hue = sum_hue/mat_add

        if sat > 5 and val > 5 and (hue < 60 or hue >= 170):
            return 'red'
        if sat > 5 and val > 5 and (70 < hue < 90):
            return 'green'
        else:
            return hue, sat, val
    
    return None

In [7]:
def draw_background_objects(map_img, map_np):
    start_time = time.time()
    global origin
    global robot_x
    global robot_y
    global visualization
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    if visualization:
        map_img[map_np == -1] = [255,255,255] #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 0] = [255,255,255]  #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 100] = [0,0,0]      #Change to [0,0,0] for color to [100] for grayscale
        origin_color = [0,0,255]              #Change to [0,0,255] for color to [0] for grayscale
        robot_color = [0,0,255]               #Change to [0,0,255] for color to [0] for grayscale
    else:
        map_img[map_np == -1] = 0    #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 0] = 0     #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 100] = 100 #Change to [0,0,0] for color to [100] for grayscale
        origin_color = 100           #Change to [0,0,255] for color to [0] for grayscale
        robot_color = 100            #Change to [0,0,255] for color to [0] for grayscale
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
#     print time.time()-start_time, "background"
    return map_img

In [8]:
def draw_ray(map_img,slices):
    
    start_time = time.time()
    
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    global areas
    global visualization
    
    slices_size = len(slices)
    
    if not display_robot_yaw:
        
        for i in range(int(slices_size)):
            try:
                
                arr = slices[i]
                appr_dist_low = (1.0-2.5*dist_error)*map_resolution*2*456.7946397925*math.pow(areas[i],-0.4705300238)              
                arr_sort = arr[arr[:,3]>appr_dist_low]
                
                # This checks whether there is an occupancy space in the slice
                
                if 100 in arr_sort[:,2]:
                    
                    arr_sort = arr_sort[np.argsort(arr_sort[:,2])]
                    first_locations = np.nonzero(np.r_[1,np.diff(arr_sort[:,2])])
                    points = np.delete(arr_sort,[2,3],axis = 1)
                    points_split = np.split(points,[first_locations[-1][-1]])
                    occ_points = points_split[1]
                    points_split = points_split[0]
                    
                    for os in range(int(len(occ_points))):
                        
                        if tags[i] == 2:
                            if visualization:
                                map_img[occ_points[os][1]][occ_points[os][0]]=[0,255,0]
                            else:
                                map_img[occ_points[os][1]][occ_points[os][0]]=30
                        
                        if tags[i] == 3: 
                            if visualization:
                                map_img[occ_points[os][1]][occ_points[os][0]]=[0,0,255]
                            else:
                                map_img[occ_points[os][1]][occ_points[os][0]]=50

                    # Set to False if you don't want to display the slices

                    if False and visualization:
                        for uos in range(int(len(points_split))):
                            map_img[points_split[uos][1]][points_split[uos][0]]=[255,0,0]
            
            except Exception as ex:
                
                print ex
                
    else:
        
        for arr in slices[-1]:
            
            map_img[arr[1]][arr[0]] = [0,255,0]
            
#     print time.time()-start_time, "draw ray"

    return map_img

In [9]:
def get_mapnbot_info(trans,robot_yaw):
    start_time = time.time()
    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    global map_np
    
    map_array = occ_map.data
    
    map_width = occ_map.info.width
    map_height = occ_map.info.height
    
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
#     print time.time()-start_time, "map/bot info"
    return map_np, robot_yaw

In [10]:
def insidetriangle((x1,x2,x3),(y1,y2,y3),map_np):
    start_time = time.time()
    global robot_x
    global robot_y
    
    xs=np.array((x1,x2,x3),dtype=float)
    ys=np.array((y1,y2,y3),dtype=float)

    # The possible range of coordinates that can be returned
    x_range=np.arange(np.min(xs),np.max(xs)+1)
    y_range=np.arange(np.min(ys),np.max(ys)+1)

    # Set the grid of coordinates on which the triangle lies. The centre of the
    # triangle serves as a criterion for what is inside or outside the triangle.
    X,Y=np.meshgrid( x_range,y_range )
    xc=np.mean(xs)
    yc=np.mean(ys)

    # From the array 'triangle', points that lie outside the triangle will be
    # set to 'False'.
    triangle = np.ones(X.shape,dtype=bool)
    for i in range(3):
        ii=(i+1)%3
        if xs[i]==xs[ii]:
            include = X *(xc-xs[i])/abs(xc-xs[i]) > xs[i] *(xc-xs[i])/abs(xc-xs[i])
        else:
            poly=np.poly1d([(ys[ii]-ys[i])/(xs[ii]-xs[i]),ys[i]-xs[i]*(ys[ii]-ys[i])/(xs[ii]-xs[i])])
            include = Y *(yc-poly(xc))/abs(yc-poly(xc)) > poly(X) *(yc-poly(xc))/abs(yc-poly(xc))
        triangle*=include

    # Output: 2 arrays with the x- and y- coordinates of the points inside the
    # triangle.
    

    x_points = X[triangle]
    y_points = Y[triangle]

    points = np.column_stack((x_points.astype(int), y_points.astype(int)))

    itbuffer = np.empty(shape=((points.size/2),4), dtype=np.int32)
    itbuffer.fill(np.nan)

    itbuffer[:,0] = x_points
    itbuffer[:,1] = y_points
    itbuffer[:,2] = map_np[points[:,1],points[:,0]]

# Get distance values    

    deltax = itbuffer[:,0]-robot_x
    deltay = itbuffer[:,1]-robot_y
    
    deltax = map_resolution*deltax
    deltay = map_resolution*deltay
    
    
    deltax = np.square(deltax)
    deltay = np.square(deltay)

    
    dist_sq = deltax+deltay
    
    dist_sq = np.sqrt(dist_sq)
#     dist_sq = map_resolution*dist_sq
    
    itbuffer[:,3] = dist_sq
#     print robot_x,robot_y
#     print np.column_stack((deltax.astype(int),deltay.astype(int)))
#     print time.time()-start_time, "inside_triangle"
    return itbuffer

In [11]:
def get_points(buoy_yaw, index):
    start_time = time.time()
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    global areas
    global dist_error
    
    appr_dist_up = (1.0+2.0*dist_error)*2*456.7946397925*math.pow(areas[index],-0.4705300238)
    point_deltax = appr_dist_up * math.sin(math.radians(buoy_yaw)) # appr_dist_up
    point_deltay = appr_dist_up * math.cos(math.radians(buoy_yaw)) #appr_dist_up
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)], dtype=int)
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)], dtype=int)

    
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))

    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))

    point3 = np.array([round(point3_x),round(point3_y)], dtype=int)
    point4 = np.array([round(point4_x),round(point4_y)], dtype=int)
    
#     print time.time()-start_time, "get_points"
    return point1,point2,point3,point4

In [12]:
def map_filter(trans, robot_yaw):
    start_time = time.time()
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
    slices = []
    i = 0
    while i < len(bearings):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if buoy_yaw < 0:
            buoy_yaw = buoy_yaw + 360
#         print bearings
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
        if display_robot_yaw and occ_map.header.seq % 5 == 0:
            bearings.append(0.0)
            print "Robot_yaw: ", robot_yaw
            print "Buoy yaw: ", buoy_yaw
            print len(slices)
##############################################################################
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw, i)    
        
#         sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)

        sliced = insidetriangle((pt3[0],pt1[0],pt4[0]),(pt3[1],pt1[1],pt4[1]),map_np)
        slices.append(np.array(sliced).astype(int))
#         print slices
        i+=1
#     print time.time()-start_time, "map_filter"
    return slices

In [13]:
def get_map(slices):
    start_time = time.time()
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    global visualization
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))
    
    #Set to 1 when publishing, to 3 for visualization
    if visualization:
        map_img_z = np.zeros((map_height,map_width,3))
    else:
        map_img_z = np.zeros((map_height, map_width))
    
    map_img_b = draw_background_objects(map_img_z,map_np)
    
    map_img_r = draw_ray(map_img_b,slices)
#     print time.time()-start_time, "get_map"
    return map_img_r

In [14]:
def create_array(map_image):
    
    global map_width
    global map_height
    global visualisation
    
    my_array = Array(data=np.array(map_image).flatten().astype(np.int32).tolist())
    
    if visualization:
         my_array.layout.dim = [Dimension("height",map_height,1*map_width*map_height),Dimension("width",map_width,1*map_width),Dimension("channel",3,3)]
    else:
        my_array.layout.dim = [Dimension("height",map_height,1*map_width*map_height),Dimension("width",map_width,1*map_width)]
    
    return my_array

In [15]:
def callback(rect, p_map):
    
    start_time = time.time()
    
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub_array
    global pub_map
    global map_height
    global map_width
    global b_list
    global visualization
    global publish_map_or_nah
    
    occ_map = p_map    
    up_occ_map = p_map
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    post = time.time()
    
    if not rospy.is_shutdown():
        
        now = rospy.Time.now()
        listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(1))
        (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
        
        euler = tf.transformations.euler_from_quaternion(rot)
        pixel_lines = map_filter(trans, math.degrees(euler[2]))
        map_image = get_map(pixel_lines)      
        
        #set to false if you want to publish the array
        #set to true if you want to publish the occupancy map
        
        if publish_map_or_nah and not visualization: 
            
            map_obj = EnvMap.EnvMap(map_image,[int(origin.y),int(origin.x)])
            buoy_data = map_obj.get_buoys()
            for item in buoy_data:
                b_list.add_buoy(item)
            map_obj.set_buoys(b_list.get_confirmed_buoys())
            map_image = map_obj.get_map()
        
            up_occ_map.data = np.array(map_image).flatten().astype(np.int32)
            if pub_map.get_num_connections > 0:
                pub_map.publish(up_occ_map)
        
        else:
            
            my_array_for_publishing = create_array(map_image)
            pub_array.publish(my_array_for_publishing)
            
        if display_rect:
            disp_rect(rect)
        if display_map and visualization:
            disp_map(map_image)
            
    print post-start_time, "transform"
    print time.time()-post, "post"
    print time.time()-start_time, "callback", "\n"

In [ ]:
if __name__ == '__main__':   
    
    global b_list
    
    b_list = Buoy_List.BuoyList()
    rect = message_filters.Subscriber("stereo/left/image_rect_color", Image)
    p_map = message_filters.Subscriber("projected_map", Map)
    
    ts = message_filters.ApproximateTimeSynchronizer([rect, p_map], 10, 0.1)
    ts.registerCallback(callback)
    rospy.spin()  

0.0500469207764 transform
0.187528133392 post
0.237602949142 callback 

0.0498368740082 transform
0.151095151901 post
0.200955867767 callback 

0.0469560623169 transform
0.138013839722 post
0.185138940811 callback 

0.0528881549835 transform
0.139139890671 post
0.192068099976 callback 

0.0499351024628 transform
0.137774944305 post
0.187726020813 callback 

0.0455958843231 transform
0.0854690074921 post
0.131227970123 callback 

0.048544883728 transform
0.0871269702911 post
0.135862827301 callback 

0.0447089672089 transform
0.0853819847107 post
0.13010597229 callback 

0.0443789958954 transform
0.0897829532623 post
0.134181976318 callback 

0.0482637882233 transform
0.0873260498047 post
0.135767936707 callback 

0.0464220046997 transform
0.136188030243 post
0.18262720108 callback 

0.0435209274292 transform
0.0844931602478 post
0.12803196907 callback 

0.0427708625793 transform
0.0858252048492 post
0.128741025925 callback 

0.0454928874969 transform
0.0872430801392 post
0.132876873016

0.0466010570526 transform
0.0847189426422 post
0.131496906281 callback 

0.0432498455048 transform
0.088198184967 post
0.131487846375 callback 

0.0438580513 transform
0.0849950313568 post
0.129112958908 callback 

0.0440008640289 transform
0.0953600406647 post
0.139487028122 callback 

0.0431399345398 transform
0.0862340927124 post
0.129399061203 callback 

0.0427749156952 transform
0.088760137558 post
0.13174200058 callback 

0.0475850105286 transform
0.0844979286194 post
0.132225036621 callback 

0.0423810482025 transform
0.0855369567871 post
0.127932071686 callback 

0.0424959659576 transform
0.0843000411987 post
0.127004861832 callback 

0.0424330234528 transform
0.0866849422455 post
0.129348993301 callback 

0.0416538715363 transform
0.08731508255 post
0.128985881805 callback 

0.0448291301727 transform
0.0844130516052 post
0.129258155823 callback 

0.0403800010681 transform
0.085629940033 post
0.126107931137 callback 

0.042083978653 transform
0.116395950317 post
0.158523082733 

0.0450179576874 transform
0.0880880355835 post
0.133140087128 callback 

0.041533946991 transform
0.0857229232788 post
0.127281904221 callback 

0.0442550182343 transform
0.0850338935852 post
0.129441976547 callback 

0.0421137809753 transform
0.0917491912842 post
0.134059906006 callback 

0.0439789295197 transform
0.0847380161285 post
0.128741979599 callback 

0.0419518947601 transform
0.089282989502 post
0.131248950958 callback 

0.0404298305511 transform
0.0869641304016 post
0.127414941788 callback 

0.0421059131622 transform
0.0860319137573 post
0.128155946732 callback 

0.0454499721527 transform
0.0868589878082 post
0.132556915283 callback 

0.0435659885406 transform
0.0889539718628 post
0.132536172867 callback 

0.0431101322174 transform
0.0869839191437 post
0.130208015442 callback 

0.0841791629791 transform
0.138597011566 post
0.223101139069 callback 

0.0410859584808 transform
0.135805130005 post
0.177016019821 callback 

0.0419249534607 transform
0.0862970352173 post
0.128235

0.044545173645 transform
0.0873439311981 post
0.131911993027 callback 

0.0435709953308 transform
0.0840859413147 post
0.12767291069 callback 

0.0473299026489 transform
0.0837559700012 post
0.131191015244 callback 

0.043566942215 transform
0.0845711231232 post
0.128384113312 callback 

0.0432150363922 transform
0.0894200801849 post
0.132826089859 callback 

0.0432679653168 transform
0.0848410129547 post
0.128139019012 callback 

0.0432970523834 transform
0.0848648548126 post
0.128247976303 callback 

0.0424599647522 transform
0.0856611728668 post
0.128149986267 callback 

0.0475060939789 transform
0.0843830108643 post
0.132029056549 callback 

0.0472459793091 transform
0.0851631164551 post
0.132663965225 callback 

0.0440459251404 transform
0.0912330150604 post
0.135298013687 callback 

0.0447010993958 transform
0.0881760120392 post
0.133027076721 callback 

0.0430138111115 transform
0.0845131874084 post
0.127544879913 callback 

0.0449311733246 transform
0.0839169025421 post
0.12890

0.0452680587769 transform
0.0896940231323 post
0.135225057602 callback 

0.046178817749 transform
0.0936250686646 post
0.139840841293 callback 

0.04727602005 transform
0.0856149196625 post
0.132936954498 callback 

0.0502750873566 transform
0.0870389938354 post
0.137418031693 callback 

0.0433740615845 transform
0.0838620662689 post
0.127394199371 callback 

0.0446808338165 transform
0.085104227066 post
0.129824876785 callback 

0.0453410148621 transform
0.0860459804535 post
0.131546020508 callback 

0.0452239513397 transform
0.0908689498901 post
0.136119842529 callback 

0.0405468940735 transform
0.0829379558563 post
0.123502969742 callback 

0.042044878006 transform
0.0839672088623 post
0.126199007034 callback 

0.0427060127258 transform
0.0835618972778 post
0.12642788887 callback 

0.0418500900269 transform
0.0849630832672 post
0.126827001572 callback 

0.0460779666901 transform
0.0893490314484 post
0.135553836823 callback 

0.0431308746338 transform
0.0884919166565 post
0.13164091

...

0.0725200176239 transform
0.14106297493 post
0.213708162308 callback 

0.0771851539612 transform
0.090488910675 post
0.167742013931 callback 

0.076052904129 transform
0.0931260585785 post
0.169201850891 callback 

0.0834989547729 transform
0.0900371074677 post
0.17355799675 callback 

0.0797090530396 transform
0.137456893921 post
0.217363119125 callback 

0.0826170444489 transform
0.0936489105225 post
0.176409006119 callback 

0.0810670852661 transform
0.138021945953 post
0.219105005264 callback 

0.0922961235046 transform
0.0902149677277 post
0.182531118393 callback 

0.0882670879364 transform
0.140022039413 post
0.228361129761 callback 

0.0923230648041 transform
0.0883150100708 post
0.180755138397 callback 

0.0918600559235 transform
0.0868258476257 post
0.178975105286 callback 

0.15460395813 transform
0.137572050095 post
0.29235291481 callback 

0.0959050655365 transform
0.0868339538574 post
0.182791948318 callback 

0.0948369503021 transform
0.0875639915466 post
0.182459831238 c

0.0853469371796 transform
0.139985084534 post
0.225347995758 callback 

0.0800759792328 transform
0.0885291099548 post
0.168679952621 callback 

0.0885992050171 transform
0.138000011444 post
0.226656198502 callback 

0.0816650390625 transform
0.116607904434 post
0.198431968689 callback 

0.0902240276337 transform
0.0898940563202 post
0.180133104324 callback 

0.078262090683 transform
0.138200998306 post
0.216479063034 callback 

0.124084949493 transform
0.138901233673 post
0.263076066971 callback 

0.0815539360046 transform
0.0868089199066 post
0.168385982513 callback 

0.0862619876862 transform
0.139400959015 post
0.225694894791 callback 

0.0822849273682 transform
0.0896880626678 post
0.171988964081 callback 

0.0777070522308 transform
0.143721103668 post
0.221629142761 callback 

0.0797698497772 transform
0.0880150794983 post
0.167977809906 callback 

0.0805389881134 transform
0.0935649871826 post
0.174346923828 callback 

0.0796158313751 transform
0.0876469612122 post
0.16727995872

0.0790050029755 transform
0.142137050629 post
0.221496105194 callback 

0.0815150737762 transform
0.0867218971252 post
0.168257951736 callback 

0.0807158946991 transform
0.138226032257 post
0.218971967697 callback 

0.0796151161194 transform
0.0927238464355 post
0.172492027283 callback 

0.0788788795471 transform
0.0898749828339 post
0.168779850006 callback 

0.0861611366272 transform
0.0866689682007 post
0.173055171967 callback 

0.0881531238556 transform
0.144022941589 post
0.232388019562 callback 

0.0932729244232 transform
0.0870070457458 post
0.180499076843 callback 

0.0877361297607 transform
0.0873880386353 post
0.175300121307 callback 

0.0878300666809 transform
0.138667106628 post
0.226644039154 callback 

0.131603002548 transform
0.138486862183 post
0.270202875137 callback 

0.105596065521 transform
0.138045787811 post
0.243658065796 callback 

0.0873579978943 transform
0.0909960269928 post
0.178514003754 callback 

0.0961580276489 transform
0.139497995377 post
0.23567891120

0.0901069641113 transform
0.139699935913 post
0.229823112488 callback 

0.0879819393158 transform
0.086886882782 post
0.174998998642 callback 

0.0935518741608 transform
0.0881671905518 post
0.18189406395 callback 

0.0791490077972 transform
0.088611125946 post
0.167783021927 callback 

0.124427080154 transform
0.150215864182 post
0.274677991867 callback 

0.0894379615784 transform
0.0865528583527 post
0.176012992859 callback 

0.077467918396 transform
0.136713027954 post
0.214199066162 callback 

0.0743670463562 transform
0.0874588489532 post
0.161958932877 callback 

0.0826461315155 transform
0.138119935989 post
0.220778942108 callback 

0.0820620059967 transform
0.0881431102753 post
0.170378923416 callback 

0.0801000595093 transform
0.0870890617371 post
0.167359113693 callback 

0.0853579044342 transform
0.08860206604 post
0.173975944519 callback 

0.0894548892975 transform
0.141603946686 post
0.23107790947 callback 

0.0884690284729 transform
0.0881938934326 post
0.176690101624 ca

0.105996847153 transform
0.0872750282288 post
0.193290948868 callback 

0.0988810062408 transform
0.087751865387 post
0.186646938324 callback 

0.106432199478 transform
0.0911099910736 post
0.197559118271 callback 

0.0964109897614 transform
0.138606071472 post
0.235076904297 callback 

0.0960290431976 transform
0.0862290859222 post
0.182276964188 callback 

0.0955529212952 transform
0.0886089801788 post
0.184181928635 callback 

